In [1]:
import requests
import psycopg2
import random
import folium
from folium.plugins import HeatMap
import openrouteservice
import time

# Create synthetic Data

In [2]:
# Bereich für zufällige Breiten- und Längengrade in Berlin Kreuzberg/Friedrichshain
latitude_range = (52.49, 52.52)
longitude_range = (13.40, 13.45)

In [3]:
# Rating-Optionen
ratings = [1, 2, 3, 4, 5]

# Funktion zur Generierung zufälliger Koordinaten innerhalb des angegebenen Bereichs
def generate_random_coordinates():
    latitude = random.uniform(*latitude_range)
    longitude = random.uniform(*longitude_range)
    return latitude, longitude

# Funktion zur Generierung eines zufälligen Ratings
def generate_random_rating():
    return random.choice(ratings)

# OpenRouteService Client initialisieren
client = openrouteservice.Client(key='5b3ce3597851110001cf624842c9deb531d34e0795400d234fd30e06')  # Ersetzen Sie 'YOUR_API_KEY' durch Ihren API-Schlüssel


In [4]:
# Funktion zur Interpolation von Punkten zwischen zwei Koordinaten
def interpolate_points(coord1, coord2, num_points):
    lat1, lon1 = coord1
    lat2, lon2 = coord2
    interpolated_points = []
    for i in range(1, num_points + 1):
        lat = lat1 + (lat2 - lat1) * i / (num_points + 1)
        lon = lon1 + (lon2 - lon1) * i / (num_points + 1)
        interpolated_points.append((lat, lon))
    return interpolated_points

# Insert Routes

In [7]:
import os
from dotenv import load_dotenv

In [8]:
load_dotenv()

True

In [15]:
def insert_route(data, start_lat, start_lon, end_lat, end_lon, rating):
    # Connect to your PostgreSQL database
    dbname = os.getenv('DATABASE_NAME')
    user = os.getenv('DATABASE_USER')
    host = os.getenv('DATABASE_HOST')
    password = os.getenv('DATABASE_PASSWORD')
    conn = psycopg2.connect(dbname=dbname, user=user, host=host, password=password)
    cur = conn.cursor()

    # Rating to weight adjustment mapping
    rating_weight_adjustment = {
        1: -2,
        2: -1,
        3: 0,
        4: 1,
        5: 2
    }

    # Insert route into the routes table with rating
    cur.execute("""
        INSERT INTO routes (start_latitude, start_longitude, end_latitude, end_longitude, rating) 
        VALUES (%s, %s, %s, %s, %s) RETURNING id
        """, (start_lat, start_lon, end_lat, end_lon, rating))
    route_id = cur.fetchone()[0]

    # Function to check if a node exists
    def get_node_id(latitude, longitude):
        cur.execute("SELECT id FROM nodes WHERE latitude = %s AND longitude = %s", (latitude, longitude))
        result = cur.fetchone()
        return result[0] if result else None

    # Insert nodes and route_nodes into the database
    sequence = 0
    coordinates = data

    # List to store node IDs in the order of the route
    node_ids = []

    for coordinate in coordinates:
        longitude, latitude = coordinate
        node_id = get_node_id(latitude, longitude)
        
        if not node_id:
            cur.execute("INSERT INTO nodes (latitude, longitude) VALUES (%s, %s) RETURNING id", (latitude, longitude))
            node_id = cur.fetchone()[0]
        
        if (route_id, node_id) not in node_ids:
            cur.execute("INSERT INTO route_way (route_id, node_id, sequence) VALUES (%s, %s, %s)", (route_id, node_id, sequence))
            sequence += 1
            node_ids.append((route_id, node_id))

    # Function to check if an edge exists and return its weight and usage_count
    def get_edge_info(node_id_start, node_id_end):
        cur.execute("SELECT weight, usage_count FROM edges WHERE node_id_start = %s AND node_id_end = %s", (node_id_start, node_id_end))
        result = cur.fetchone()
        return result if result else None

    # Function to update edge weight and usage_count
    def update_edge(node_id_start, node_id_end, weight_adjustment):
        cur.execute("UPDATE edges SET weight = weight + %s, usage_count = usage_count + 1 WHERE node_id_start = %s AND node_id_end = %s", 
                    (weight_adjustment, node_id_start, node_id_end))

    # Insert edges into the database
    for i in range(len(node_ids) - 1):
        start_node_id = node_ids[i][1]
        end_node_id = node_ids[i + 1][1]
        weight_adjustment = rating_weight_adjustment[rating]
        
        edge_info = get_edge_info(start_node_id, end_node_id)
        
        if edge_info is None:
            cur.execute("INSERT INTO edges (node_id_start, node_id_end, weight, usage_count) VALUES (%s, %s, %s, %s)", 
                        (start_node_id, end_node_id, weight_adjustment, 1))
        else:
            update_edge(start_node_id, end_node_id, weight_adjustment)

    # Commit the transaction
    conn.commit()

    # Close the cursor and connection
    cur.close()
    conn.close()

# Call random Entries

In [16]:
num_entries = 200

for i in range(num_entries):
    # Generieren der zufälligen Start- und Endkoordinaten
    start_lat, start_lon = generate_random_coordinates()
    end_lat, end_lon = generate_random_coordinates()
    
    # Generieren des zufälligen Ratings
    rating = generate_random_rating()

    # Routeninformationen von OpenRouteService abrufen
    response = client.directions(coordinates=[[start_lon, start_lat], [end_lon, end_lat]], profile='driving-car', format='geojson')
    data = response['features'][0]['geometry']['coordinates']

    insert_route(data, start_lat, start_lon, end_lat, end_lon, rating)

    print(data)

    time.sleep(1) #Use sleep timer, because otherwise the API will block you after aroung 50 requests

[[13.41033, 52.514966], [13.410383, 52.515177], [13.410605, 52.516057], [13.410763, 52.516069], [13.412325, 52.515834], [13.41283, 52.515762], [13.413031, 52.515701], [13.413619, 52.515679], [13.415027, 52.515753], [13.415158, 52.51576], [13.416166, 52.51577], [13.416425, 52.515747], [13.416873, 52.515694], [13.417438, 52.51559], [13.417996, 52.515468], [13.418174, 52.515428], [13.418858, 52.515234], [13.41904, 52.51517], [13.419413, 52.515042], [13.419455, 52.515028], [13.419585, 52.514983], [13.419644, 52.514963], [13.419946, 52.514859], [13.421579, 52.514298], [13.421848, 52.514206], [13.422242, 52.51407], [13.422349, 52.514033], [13.422462, 52.513992], [13.422592, 52.513947], [13.423079, 52.513785], [13.423341, 52.513694], [13.423855, 52.51351], [13.424029, 52.513433], [13.424138, 52.513372], [13.424372, 52.513245], [13.424834, 52.512961], [13.425283, 52.512645], [13.425841, 52.512261], [13.426364, 52.511969], [13.426398, 52.51195], [13.426885, 52.511696], [13.427484, 52.511387], [

c:\Users\andyu\OneDrive\Počítač\Enterprise Architectures for Big Data\Big_Data_Project\big_data_venv\Lib\site-packages\openrouteservice\client.py:211: UserWarning: Rate limit exceeded. Retrying for the 1st time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
c:\Users\andyu\OneDrive\Počítač\Enterprise Architectures for Big Data\Big_Data_Project\big_data_venv\Lib\site-packages\openrouteservice\client.py:211: UserWarning: Rate limit exceeded. Retrying for the 2nd time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
c:\Users\andyu\OneDrive\Počítač\Enterprise Architectures for Big Data\Big_Data_Project\big_data_venv\Lib\site-packages\openrouteservice\client.py:211: UserWarning: Rate limit exceeded. Retrying for the 3rd time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
c:\Users\andyu\OneDrive\Počítač\Enterprise Architectures for Big Data\Big_Data_

[[13.424342, 52.510556], [13.424297, 52.510509], [13.424228, 52.510483], [13.424112, 52.510474], [13.42372, 52.509844], [13.423675, 52.509775], [13.423311, 52.509197], [13.423427, 52.50916], [13.4246, 52.508833], [13.424672, 52.508813], [13.425106, 52.508663], [13.425382, 52.508547], [13.425692, 52.508426], [13.426672, 52.508024], [13.427522, 52.507652], [13.427712, 52.507571], [13.428259, 52.507353], [13.428301, 52.507948], [13.428361, 52.508138], [13.428437, 52.508261], [13.428805, 52.508688], [13.429018, 52.508933], [13.429519, 52.509494], [13.429708, 52.509697], [13.430028, 52.510057], [13.430113, 52.510193], [13.43071, 52.509971], [13.431713, 52.509599], [13.431924, 52.509521], [13.4322, 52.509419], [13.432404, 52.509343], [13.433296, 52.509007], [13.433381, 52.508976], [13.433908, 52.50878], [13.434632, 52.508457], [13.435009, 52.508256], [13.435402, 52.508007], [13.436081, 52.507483], [13.436749, 52.506954], [13.436871, 52.507013], [13.437148, 52.507165], [13.437912, 52.507683],

c:\Users\andyu\OneDrive\Počítač\Enterprise Architectures for Big Data\Big_Data_Project\big_data_venv\Lib\site-packages\openrouteservice\client.py:211: UserWarning: Rate limit exceeded. Retrying for the 7th time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,


[[13.410379, 52.497843], [13.41062, 52.497853], [13.410684, 52.497353], [13.410691, 52.497308], [13.410708, 52.497183], [13.410732, 52.497008], [13.410823, 52.496843], [13.410953, 52.496736], [13.411092, 52.496665], [13.41126, 52.496603], [13.411744, 52.496463], [13.411863, 52.496459], [13.411952, 52.496503], [13.411975, 52.496531], [13.41206, 52.496648], [13.412081, 52.496678], [13.41223, 52.496883], [13.412253, 52.496915], [13.412278, 52.49695], [13.412334, 52.497028], [13.412699, 52.497557], [13.412813, 52.497712], [13.412896, 52.497815], [13.413014, 52.497853], [13.41318, 52.497857], [13.413478, 52.49789], [13.413749, 52.497955], [13.414186, 52.497834], [13.414513, 52.497746], [13.414568, 52.497731], [13.415706, 52.497424], [13.415763, 52.497409], [13.415789, 52.497402], [13.416111, 52.497315], [13.416319, 52.497259], [13.416328, 52.497196], [13.416381, 52.497167], [13.41649, 52.497172], [13.416538, 52.497207], [13.416526, 52.497273], [13.416467, 52.497301], [13.416542, 52.497405],

# Map Creation

In [25]:
import psycopg2
import folium
import matplotlib.colors as mcolors
import matplotlib.cm as cm

# Funktion zur Bestimmung der Farbe basierend auf dem Gewicht
def get_color(weight, vmin, vmax):
    norm = mcolors.TwoSlopeNorm(vmin=vmin, vcenter=0, vmax=vmax)
    cmap = cm.RdYlGn  # Direkter Zugriff auf die Colormap
    return mcolors.to_hex(cmap(norm(weight)))

# Verbindung zur PostgreSQL-Datenbank
conn = psycopg2.connect("dbname='bike_project' user='postgres' host='localhost' password='123456'")
cur = conn.cursor()

# Abfrage aller Edges mit deren Start- und Endkoordinaten sowie Gewichten
cur.execute("""
    SELECT ns.latitude, ns.longitude, ne.latitude, ne.longitude, e.weight
    FROM edges e
    JOIN nodes ns ON e.node_id_start = ns.id
    JOIN nodes ne ON e.node_id_end = ne.id
""")
edges = cur.fetchall()

# Bestimmen des minimalen und maximalen Gewichts
weights = [edge[4] for edge in edges]
vmin = min(weights)
vmax = max(weights)

# Karte initialisieren
map_center = [52.5, 13.4]  # Beispielzentrum (Berlin)
mymap = folium.Map(location=map_center, zoom_start=12)

# Edges zur Karte hinzufügen
for edge in edges:
    start_lat, start_lon, end_lat, end_lon, weight = edge
    color = get_color(weight, vmin, vmax)
    folium.PolyLine(
        locations=[(start_lat, start_lon), (end_lat, end_lon)],
        color=color,
        weight=5,
        opacity=0.8
    ).add_to(mymap)

# HTML-Datei speichern
mymap.save('route_map.html')

# Verbindung schließen
cur.close()
conn.close()
